In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.extend([ '../', '../../', '../edge_pred/'])
import torch
import torch.nn as nn
import torch.optim as optim
from data import input_data
import dataloader as dl
from args import Args
from model2 import Het_En, EdgePredictor, Het_classify, encoder2
from smote import oversample
from train_edge_pred2 import train_smote, test_smote
import cProfile

# Set device to GPU if available, else use CPU
args = Args()
args.aminer_train()
torch.cuda.empty_cache()

Current device: NVIDIA A100 80GB PCIe


In [2]:
# data = input_data(args) 
# torch.save(data, 'data.pt')
data = torch.load('../edge_pred/edge_pred_data.pt')
print(data)

HeteroData(
  a={
    num_nodes=20171,
    y=[20171]
  },
  p={ num_nodes=13250 },
  v={ num_nodes=18 },
  p_title_embed={ x=[13250, 128] },
  p_abstract_embed={ x=[13250, 128] },
  p_net_embed={ x=[13250, 128] },
  p_a_net_embed={ x=[13250, 128] },
  p_v_net_embed={ x=[13250, 128] },
  a_net_embed={ x=[20171, 128] },
  a_text_embed={ x=[20171, 128] },
  v_net_embed={ x=[18, 128] },
  v_text_embed={ x=[18, 128] },
  e={
    x=[29166, 2],
    y=[29166]
  },
  (a, walk, a)={ edge_index=[2, 90641] },
  (a, walk, p)={ edge_index=[2, 89734] },
  (a, walk, v)={ edge_index=[2, 28573] },
  (p, walk, a)={ edge_index=[2, 65555] },
  (p, walk, p)={ edge_index=[2, 66190] },
  (p, walk, v)={ edge_index=[2, 19461] },
  (v, walk, a)={ edge_index=[2, 90] },
  (v, walk, p)={ edge_index=[2, 90] },
  (v, walk, v)={ edge_index=[2, 13] }
)


In [3]:
print(data['a','walk','p'].edge_index)
device = args.device
# data = data.to(device)

# Send all x tensors to the device
data['p_title_embed']['x'] = data['p_title_embed']['x'].to(device)
data['p_abstract_embed']['x'] = data['p_abstract_embed']['x'].to(device)
data['p_net_embed']['x'] = data['p_net_embed']['x'].to(device)
data['p_a_net_embed']['x'] = data['p_a_net_embed']['x'].to(device)
# data['p_p_net_embed']['x'] = data['p_p_net_embed']['x'].to(device)
data['p_v_net_embed']['x'] = data['p_v_net_embed']['x'].to(device)
data['a_net_embed']['x'] = data['a_net_embed']['x'].to(device)
data['a_text_embed']['x'] = data['a_text_embed']['x'].to(device)
data['v_net_embed']['x'] = data['v_net_embed']['x'].to(device)
data['v_text_embed']['x'] = data['v_text_embed']['x'].to(device)

# Send all y tensors to the device
data['a']['y'] = data['a']['y'].to(device)

# Send all edge_index tensors to the device
data['a', 'walk', 'a']['edge_index'] = data['a', 'walk', 'a']['edge_index'].to(device)
data['a', 'walk', 'p']['edge_index'] = data['a', 'walk', 'p']['edge_index'].to(device)
data['a', 'walk', 'v']['edge_index'] = data['a', 'walk', 'v']['edge_index'].to(device)
data['p', 'walk', 'a']['edge_index'] = data['p', 'walk', 'a']['edge_index'].to(device)
data['p', 'walk', 'p']['edge_index'] = data['p', 'walk', 'p']['edge_index'].to(device)
data['p', 'walk', 'v']['edge_index'] = data['p', 'walk', 'v']['edge_index'].to(device)
data['v', 'walk', 'a']['edge_index'] = data['v', 'walk', 'a']['edge_index'].to(device)
data['v', 'walk', 'p']['edge_index'] = data['v', 'walk', 'p']['edge_index'].to(device)
data['v', 'walk', 'v']['edge_index'] = data['v', 'walk', 'v']['edge_index'].to(device)

edge_indices = [ data['a', 'walk', 'a'].edge_index, data['a', 'walk', 'p'].edge_index, data['a', 'walk', 'v'].edge_index ]

tensor([[    0,     0,     0,  ..., 20169, 20169, 20170],
        [10523, 10550, 10395,  ...,  9100,  9274,  4378]])


In [4]:
c_train_num = dl.train_num(data['a'].y, args.im_class_num, args.class_samp_num[0], args.im_ratio)
print(c_train_num, sum(c_train_num))
train_idx, val_idx, test_idx, c_num_mat = dl.segregate(data['a'].y, c_train_num, args)
print("train_idx: ", train_idx, len(train_idx))
print("val_idx: ", val_idx, len(val_idx))
print("test_idx: ", test_idx, len(test_idx))
# print(c_num_mat)

[50, 30, 25, 20] 125
0 335
1 202
2 299
3 289
train_idx:  [14147, 1051, 17095, 7503, 7936, 17118, 2931, 421, 1195, 8409, 10422, 18801, 18004, 3191, 11979, 16139, 13109, 8058, 19059, 16001, 3426, 4572, 11803, 355, 5652, 4945, 18712, 7370, 5582, 4863, 7900, 17721, 7909, 9359, 10535, 15393, 3649, 19625, 3700, 11747, 6896, 3208, 3042, 17258, 9286, 6833, 8423, 7481, 18876, 5208, 9795, 4300, 18214, 15705, 5324, 8721, 1730, 17796, 13892, 7082, 11659, 9834, 12172, 17063, 19102, 11519, 11560, 16191, 6769, 590, 9688, 5002, 14009, 8272, 16326, 11823, 19784, 15679, 5488, 153, 16344, 1538, 13082, 12515, 9363, 10151, 5932, 9930, 2034, 992, 6010, 14625, 15042, 13097, 5404, 12562, 17924, 3404, 19663, 10918, 8769, 6670, 17328, 13689, 13157, 16631, 1260, 10506, 16098, 9040, 11291, 8711, 12810, 9364, 14614, 10229, 3238, 3461, 17944, 5739, 8630, 7484, 17881, 16685, 12603] 125
val_idx:  [5782, 10442, 21, 10301, 1135, 14500, 16033, 13395, 19079, 15984, 5254, 295, 11727, 7688, 10865, 20158, 489, 14025, 4553, 

In [5]:
# os_mode = 'up'
# portion = sum(c_train_num)/(4*c_train_num[1])
# if os_mode != 'reweight' and os_mode != 'no':
#     if portion >= 1:
#         args.node_dim[0] = args.A_n + (portion-1)*c_train_num[1] + int((portion - int(portion))*c_train_num[1])
#     else:
#         args.node_dim[0] = args.A_n + int((portion - int(portion))*c_train_num[1])

# args.node_dim[0] += 22

In [6]:
encoder = Het_En(args.embed_dim, args.dropout)
encoder2 = encoder2(args.embed_dim, args.dropout)
classifier = Het_classify(args.embed_dim, args.nclass, args.dropout)
decoder_a = EdgePredictor(args.embed_dim)
decoder_p = EdgePredictor(args.embed_dim)
decoder_v = EdgePredictor(args.embed_dim)
decoder_list = [decoder_a, decoder_p, decoder_v]

decoder_pa = EdgePredictor(args.embed_dim)
decoder_pp = EdgePredictor(args.embed_dim)
decoder_pv = EdgePredictor(args.embed_dim)
decoder_list2 = [decoder_pa, decoder_pp, decoder_pv]
#print(features.shape)
encoder.to(device)
encoder2.to(device)
classifier.to(device)
for decoder in decoder_list:
    decoder.to(device)
for decoder in decoder_list2:
    decoder.to(device)

## Training Part

In [7]:
torch.cuda.empty_cache()
# profiler = cProfile.Profile()
# profiler.enable()
train_smote(data, edge_indices, encoder, encoder2, classifier, decoder_list, decoder_list2,
            train_idx, val_idx, test_idx, args, os_mode = 'gsm', train_mode = 'preo')
# profiler.disable()

Epoch [1/200], Loss: 1.4189, Accuracy: 0.2063, Edge Accuracy: ([0.9998607685361991, 0.9998768491660454, 0.9999972457708812], [0.9997812327243577, 0.9997821003915984, 0.9448385744234801])
Class 0: AUC-ROC- 0.6183, F1 Score- 0.0000; Class 1: AUC-ROC- 0.4537, F1 Score- 0.3949; Class 2: AUC-ROC- 0.5576, F1 Score- 0.0000; Class 3: AUC-ROC- 0.4231, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.5132; Macro-Average F1 Score: 0.0987
Validation Loss: 1.3859, Validation Accuracy: 0.2773, Validation Edge Accuracy: ([], [])
Class 0: AUC-ROC- 0.6966, F1 Score- 0.0000; Class 1: AUC-ROC- 0.5104, F1 Score- 0.4000; Class 2: AUC-ROC- 0.6396, F1 Score- 0.0000; Class 3: AUC-ROC- 0.3675, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.5535; Macro-Average F1 Score: 0.1000
Test Loss: 1.3843, Test Accuracy: 0.3100, Test Edge Accuracy: ([], [])
Class 0: AUC-ROC- 0.7095, F1 Score- 0.0000; Class 1: AUC-ROC- 0.5742, F1 Score- 0.4000; Class 2: AUC-ROC- 0.7345, F1 Score- 0.0000; Class 3: AUC-ROC- 0.3668, F1 Score- 0.0000; M

In [8]:
# profiler.print_stats(sort='cumulative')  # Print results to the console
# # Or save to a file
# with open('profile_results.txt', 'w') as f:
#     profiler.dump_stats(f)

In [9]:
test_smote(data, edge_indices, encoder, encoder2, classifier, decoder_list, decoder_list2,
           test_idx, args = args, dataset = 'Test', os_mode = 'no', train_mode = 'preo')

Test Loss: 1.0222, Test Accuracy: 0.9767, Test Edge Accuracy: ([], [])
Class 0: AUC-ROC- 0.9999, F1 Score- 0.9804; Class 1: AUC-ROC- 0.9988, F1 Score- 0.8621; Class 2: AUC-ROC- 0.9918, F1 Score- 0.9542; Class 3: AUC-ROC- 0.9993, F1 Score- 0.9583; Macro-Average AUC-ROC: 0.9974; Macro-Average F1 Score: 0.9388


(0.9766666666666667,
 0.9974370370370369,
 0.938760705431598,
 [0.9999407407407407,
  0.9987555555555555,
  0.991762962962963,
  0.9992888888888889],
 [0.9803921568627451,
  0.8620689655172413,
  0.954248366013072,
  0.9583333333333334])

In [10]:
torch.save(decoder_list2[0], 'pre_decoder/new_edge_pred/decoder_pa.pth')
torch.save(decoder_list2[1], 'pre_decoder/new_edge_pred/decoder_pp.pth')
torch.save(decoder_list2[2], 'pre_decoder/new_edge_pred/decoder_pv.pth')
torch.save(encoder2, 'pre_decoder/new_edge_pred/encoder2.pth')
torch.save(encoder, 'pre_decoder/new_edge_pred/encoder.pth')